In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import figure
import random as rd
import copy

In [2]:
class Independent_Cascade():
    def __init__(self):
        self.g = nx.DiGraph()

    def fit(self, g):
        # fit graph with probability
        out_degree = g.out_degree(weight='None')
        max_out_degree = max(out_degree, key=lambda item:item[1])
        for e in g.edges():
            if(out_degree[e[0]] >= 10):
                # g[e[0]][e[1]]['probability'] = 1 - out_degree[e[0]]/max_out_degree[1]
                g[e[0]][e[1]]['probability'] = 1 / int(np.log(out_degree[e[0]]))
            else:
                g[e[0]][e[1]]['probability'] = 1
        self.g = g
        return g
        
    # diffusion to all possible nodes
    def diffusion_all(self, seed_nodes):
        if(seed_nodes == []):
            return [], []
        activated_nodes = copy.deepcopy(seed_nodes)
        old_activated_nodes = seed_nodes
        activate_nums = [len(activated_nodes)]
        while(True):
            new_activated_nodes = []
            for node in old_activated_nodes:
                for predecessors  in self.g.predecessors(node):
                    if( predecessors in activated_nodes):
                        continue
                    if self.g[predecessors][node]['probability'] >= rd.random():
                        new_activated_nodes.append(predecessors)
                activated_nodes.extend(new_activated_nodes)
            if len(new_activated_nodes) == 0:
                break
            old_activated_nodes = new_activated_nodes
            activate_nums.append(len(new_activated_nodes))
        return activated_nodes, activate_nums

    # diffusion to max step
    def diffusion_step(self, seed_nodes, max_step=1):
        if(seed_nodes == []):
            return [], []
        activated_nodes = copy.deepcopy(seed_nodes)
        old_activated_nodes = seed_nodes
        activate_nums = [len(activated_nodes)]
        for _ in range(max_step):
            new_activated_nodes = []
            for node in old_activated_nodes:
                for predecessors  in self.g.predecessors(node):
                    if( predecessors in activated_nodes):
                        continue
                    if self.g[predecessors][node]['probability'] >= rd.random():
                        new_activated_nodes.append(predecessors)
                activated_nodes.extend(new_activated_nodes)
            if len(new_activated_nodes) == 0:
                break
            old_activated_nodes = new_activated_nodes
            activate_nums.append(len(new_activated_nodes))
        return activated_nodes, activate_nums

In [3]:
# read from gml
G = nx.read_gml('PB2020.gml')

In [4]:
model = Independent_Cascade()
model.fit(G)

In [5]:
activated_nodes, activate_nums = model.diffusion_all(['enrique_tarrio',
 'GavinM_ProudBoy',
 'proudboy2012',
 'Premises187',
 'ProudBoysGBG',
 'principe_giovan',
 'MoralDK'])
print(activate_nums)

[7, 1439, 9, 228, 1]


In [6]:
sample = rd.sample(list(G.nodes), 10)
activated_nodes, activate_nums = model.diffusion_all(sample)
print(activate_nums)

[10, 1, 357]


In [25]:
sample = rd.sample(list(G.nodes), 10)
activated_nodes, activate_nums = model.diffusion_step(sample, 1)
print(activate_nums)

[10, 2]
